# Generating SQL: SQLAlchemy vs SQLGlot

- 1.4 func docs: https://docs.sqlalchemy.org/en/14/core/functions.html
- 1.4 func tutorial: https://docs.sqlalchemy.org/en/14/tutorial/data_select.html#tutorial-functions
- dialects: https://docs.sqlalchemy.org/en/20/dialects/
- dialect load https://stackoverflow.com/questions/53284762/nosuchmoduleerror-cant-load-plugin-sqlalchemy-dialectssnowflake


SQAlchemy pros:

- already a dependency so no need to add a new one
- better sql manipulation for composing queries (since it's a Python API) than using jinja2

sqlalchemy cons:

- recently released sqlalchemy 2.0 breaks the API. since many people haven't upgraded, we'll have to support 1.4 and 2.0
- harder to contribute (contributors need to learn sqlalchemy)
- need to declare column types manually (not sure about this one, maybe it isn't needed)
- complex data types (such as arrays) not supported

sqlglot pros:

- supports complex data types (such as arrays) - although we have to add some tests to ensure it produces valid SQL

sqlglot cons:

- we need to add an extra dependency (although we could drop sqlparse in favor of sqlglot)


conclusion: use sqlglot

In [19]:
import sqlalchemy as db
from sqlalchemy import select, func
from sqlalchemy.dialects import postgresql, oracle, sqlite, mysql

db.__version__

'1.4.46'

Step 1. create a tabe object

In [4]:
t = db.Table("a", db.MetaData(), db.Column('y', db.Integer), db.Column('x', db.Integer))

to generate sql, call `table.select`

In [20]:
print(t.select(t.columns.y).where(t.columns.y == 2).group_by(t.columns.y))

SELECT a.y, a.x 
FROM a 
WHERE a.y AND a.y = :y_1 GROUP BY a.y


we can also call `sqlalchemy.select` (unsure what the difference is):

In [21]:
print(db.select(t.columns.y, db.func.sum(t.columns.x)).where(t.columns.y == 1).group_by(t.columns.y))

SELECT a.y, sum(a.x) AS sum_1 
FROM a 
WHERE a.y = :y_1 GROUP BY a.y


more info on generating sql here: https://stackoverflow.com/a/23835766/709975

sql alchemy only translates basic functions depending on the dialect, e.g., it'll call the right function to get the current timestamp.

it also applies the right column/table name identifier (double quotes vs backticks):

In [26]:
q = select(t.columns.y.label("some thing"), func.percentile_disc(0.25).within_group(t.columns.y), func.now())

print(q.compile(dialect=postgresql.dialect(), compile_kwargs={"literal_binds": True}))

SELECT a.y AS "some thing", percentile_disc(0.25) WITHIN GROUP (ORDER BY a.y) AS anon_1, now() AS now_1 
FROM a


In [27]:
print(q.compile(dialect=mysql.dialect(), compile_kwargs={"literal_binds": True}))

SELECT a.y AS `some thing`, percentile_disc(0.25) WITHIN GROUP (ORDER BY a.y) AS anon_1, now() AS now_1 
FROM a


In [28]:
print(q.compile(dialect=oracle.dialect(), compile_kwargs={"literal_binds": True}))

SELECT a.y AS "some thing", percentile_disc(0.25) WITHIN GROUP (ORDER BY a.y) AS anon_1, CURRENT_TIMESTAMP AS now_1 
FROM a


we can also pass engine objects:

In [30]:
engine = db.create_engine("duckdb://")

In [32]:
print(q.compile(engine, compile_kwargs={"literal_binds": True}))

SELECT a.y AS "some thing", percentile_disc(0.25) WITHIN GROUP (ORDER BY a.y) AS anon_1, now() AS now_1 
FROM a


problem! it requires manually declaration of data types:

In [33]:
q = select(t.columns.y.label("some thing"), func.percentile_disc([0.25, 0.5]).within_group(t.columns.y), func.now())
try:
    q.compile(compile_kwargs={"literal_binds": True})
except Exception as e:
    print(e)

No literal value renderer is available for literal value "[0.25, 0.5]" with datatype NULL


also, I couldn't find an API to load plugins:

In [26]:
from sqlalchemy.dialects import plugins

In [34]:
try:
    plugins.load("snowflake")
except Exception as e:
    print(e)

name 'plugins' is not defined


In [38]:
from sqlalchemy.dialects import registry

# found this on stack overflow
registry.register('snowflake', 'snowflake.sqlalchemy', 'dialect')
snowflake = registry.impls["snowflake"]()()

we can also load them directly but this might depend on the package structure ?

In [39]:
from duckdb_engine import Dialect
duckdb = Dialect()

In [40]:
print(select(func.now()).compile(dialect=snowflake))

SELECT now() AS now_1


In [41]:
print(select(func.now()).compile(dialect=duckdb))

SELECT now() AS now_1


## using sqlglot

In [2]:
import sqlglot

query = """

SELECT
percentile_disc(0.25) WITHIN GROUP (ORDER BY "somecolumn") AS q1,
percentile_disc(0.50) WITHIN GROUP (ORDER BY "somecolumn") AS med,
percentile_disc(0.75) WITHIN GROUP (ORDER BY "somecolumn") AS q3,
AVG("somecolumn") AS mean,
COUNT(*) AS N
FROM "sometable"
"""

print(sqlglot.transpile(query, read="duckdb", write="mysql")[0])

SELECT PERCENTILE_DISC(0.25) WITHIN GROUP (ORDER BY `somecolumn`) AS q1, PERCENTILE_DISC(0.50) WITHIN GROUP (ORDER BY `somecolumn`) AS med, PERCENTILE_DISC(0.75) WITHIN GROUP (ORDER BY `somecolumn`) AS q3, AVG(`somecolumn`) AS mean, COUNT(*) AS N FROM `sometable`


data types supported, is this correct?

In [3]:
query = """

SELECT
percentile_disc([0.25, 0.5, 0.75]) WITHIN GROUP (ORDER BY "somecolumn")
FROM "sometable"
"""

# percentile disk doesn't exist on mysql
print(sqlglot.transpile(query, read="duckdb", write="mysql")[0])

SELECT PERCENTILE_DISC(ARRAY(0.25, 0.5, 0.75)) WITHIN GROUP (ORDER BY `somecolumn`) FROM `sometable`
